# Automated sunspots detection based on morphological operators
This notebook consists of an algorithm based on mathematical morphology operators to segment sunspots on H-Alpha images acquired by the spectroheliograph of Coimbra Observatory (OGAUC) and AIA 1600 images captured by the Solar Dynamics Observatory mission (SDO).

The set available to test the algorithm consists of two pairs of images of these sensors from the same day. For detecting the sunspots in each image use the following names as 'img_in' in Step 1 ('Reading and displaying the input image):

2014oct23_CN_OGAUC <br> 
2014nov29_CN_OGAUC <br><br>
2014oct23_AIA_SDO <br> 
2014nov29_AIA_SDO <br>

Some of the parameters of the algorithm may need to be fine-tuned in each image for obtaining a better result.


## 0. Preparation 
Importing the necessary modules and defining the size of the window to display the images.

In [ ]:
import diplib as dip
import matplotlib as plt
plt.rcParams["figure.figsize"] = (12,10)

## 1. Reading and displaying the input image 
To test the algorithm with one of the 4 images mentioned above, just insert the name of the file in 'img_in': 

In [ ]:
img_in = '2014oct23_CN_OGAUC'
img = dip.ImageReadTIFF(img_in)
img.Show()

## 2. Creating the sun mask
Pre-processing to filter the white text at the top and bottom of the image using an opening with a structuring element disk of size 'se1':

In [ ]:
se1 = 1
ope = dip.Opening(img,se1)
ope.Show()

Thresholding of the opened image to obtain a binary mask of the sun (correspond to all non-zero pixels of the image):

In [ ]:
t1 = 1
bin = dip.FixedThreshold(ope,t1)
bin.Show()

Filling the holes (spots that might have the same intensity of the background) with an operator based on geodesic dilation:

In [ ]:
bin = dip.FillHoles(bin)
bin.Show()

Contour of the mask, obtained through the morphological gradient, superimposed to the input image:

In [ ]:
contSun = dip.MorphologicalGradientMagnitude(bin)
img_ovr_sun = dip.Overlay(img,contSun)
img_ovr_sun.Show()

## 3. Spots Segmentation (umbra + penumbra)
Detection of the sunspots with the black tophat transform, obatined with a closing operator with a disk of size 'se2' as structuring element: 

In [ ]:
se2 = 1
BTH = dip.Closing(ope,se2) - ope
BTH.Show()

The binarization of the sunspots is obtained through a thresholding of the black top hat:

In [ ]:
t2 = 1
BTHbin = dip.FixedThreshold(BTH,t2)
BTHbin.Show()

The filtering by opening-reconstruction removes very small objects that are unlike to be sunspots:

In [ ]:
se3 = 1
fBTHbin = dip.OpeningByReconstruction(BTHbin,se3)
fBTHbin.Show()

Determination of the contours of the sunspots with a half-gradient (or morphological external gradient), superimposed to the input image:

In [ ]:
contSPOTS = dip.Dilation(fBTHbin) - fBTHbin
img_ovr_SPOTS = dip.Overlay(img,contSPOTS)
img_ovr_SPOTS.Show()

Save the ouput result in a standard TIFF format:

In [ ]:
img_out = img_in + '_spots.tif'
dip.ImageWriteTIFF(img_ovr_SPOTS,img_out)